In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
        .appName("comm") \
        .appName("P1") \
        .getOrCreate()

In [5]:
import os

path, dirs, files = next(os.walk("./Data/FiveYears/"))
file_count = len(files)
print("No of Files merged are:")
print(file_count)

dataframes_list = []
for i in range(file_count):
    temp_df = spark.read.csv("./Data/FiveYears/"+files[i], header = True)
    academmicYear = files[i]
    dataframes_list.append(temp_df)
    print(academmicYear[6:10])
for dataset in dataframes_list:
    print(dataset.count())


No of Files merged are:
5
2015
2017
2019
2016
2018
7666
7112
6694
7238
6807


In [8]:
dataframes_list[0].count()

7666

In [6]:
dataframes_list[0].createOrReplaceTempView("temp_2015")
sqlDF_2015 = spark.sql("SELECT 2015 as Academic_Year, * FROM temp_2015 ")

dataframes_list[3].createOrReplaceTempView("temp_2016")
sqlDF_2016 = spark.sql("SELECT 2016 as Academic_Year, * FROM temp_2016 ")

dataframes_list[1].createOrReplaceTempView("temp_2017")
sqlDF_2017 = spark.sql("SELECT 2017 as Academic_Year, * FROM temp_2017 ")

dataframes_list[4].createOrReplaceTempView("temp_2018")
sqlDF_2018 = spark.sql("SELECT 2018 as Academic_Year, * FROM temp_2018 ")

dataframes_list[2].createOrReplaceTempView("temp_2019")
sqlDF_2019 = spark.sql("SELECT 2019 as Academic_Year, * FROM temp_2019 ")


In [9]:
print(sqlDF_2015.count())
print(sqlDF_2016.count())
print(sqlDF_2017.count())
print(sqlDF_2018.count())
print(sqlDF_2019.count())

7666
7238
7112
6807
6694


In [10]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [11]:
mergeddf = unionAll( sqlDF_2015,sqlDF_2016,sqlDF_2017, sqlDF_2018, sqlDF_2019)

In [12]:
print('Merged dataset with all data')
print(mergeddf.count()) # 74230

Merged dataset with all data
35517


In [13]:
sqlDF_2015.createOrReplaceTempView("temp_2015")
sqlDF_2017.createOrReplaceTempView("temp_2017")
sqlDF_2019.createOrReplaceTempView("temp_2019")
sqlDF_2016.createOrReplaceTempView("temp_2016") 
sqlDF_2018.createOrReplaceTempView("temp_2018") 

In [14]:
#OPEID, INSTNM 5 years
sqlDF_MergedWithSchool = spark.sql("SELECT distinct temp_2015.OPEID, temp_2015.INSTNM FROM temp_2015 inner join temp_2016 on temp_2015.OPEID=temp_2016.OPEID inner join temp_2017 on temp_2015.OPEID=temp_2017.OPEID inner join temp_2018 on temp_2015.OPEID=temp_2018.OPEID inner join temp_2019 on temp_2015.OPEID=temp_2019.OPEID ")

In [15]:
sqlDF_MergedWithSchool.count() #Distinct college in every year are 5186 

6086

In [16]:
# Union all with distinct school names 
mergeddf.createOrReplaceTempView("temp_mergeddf")
sqlDF_MergedWithSchool.createOrReplaceTempView("temp_MergedWithSchool")

In [17]:
sqlDF_SchoolInEveryYear = spark.sql("SELECT * FROM temp_mergeddf where OPEID in (select OPEID from temp_MergedWithSchool)")

In [18]:
type(sqlDF_SchoolInEveryYear)

pyspark.sql.dataframe.DataFrame

In [19]:
sqlDF_SchoolInEveryYear.count()

30445

In [20]:
sqlDF_SchoolInEveryYear.cache()#7:14

DataFrame[Academic_Year: int, UNITID: string, OPEID: string, OPEID6: string, INSTNM: string, CITY: string, STABBR: string, ZIP: string, ACCREDAGENCY: string, INSTURL: string, NPCURL: string, SCH_DEG: string, HCM2: string, MAIN: string, NUMBRANCH: string, PREDDEG: string, HIGHDEG: string, CONTROL: string, ST_FIPS: string, REGION: string, LOCALE: string, LOCALE2: string, LATITUDE: string, LONGITUDE: string, CCBASIC: string, CCUGPROF: string, CCSIZSET: string, HBCU: string, PBI: string, ANNHI: string, TRIBAL: string, AANAPII: string, HSI: string, NANTI: string, MENONLY: string, WOMENONLY: string, RELAFFIL: string, ADM_RATE: string, ADM_RATE_ALL: string, SATVR25: string, SATVR75: string, SATMT25: string, SATMT75: string, SATWR25: string, SATWR75: string, SATVRMID: string, SATMTMID: string, SATWRMID: string, ACTCM25: string, ACTCM75: string, ACTEN25: string, ACTEN75: string, ACTMT25: string, ACTMT75: string, ACTWR25: string, ACTWR75: string, ACTCMMID: string, ACTENMID: string, ACTMTMID: str

In [21]:
sqlDF_SchoolInEveryYear.createOrReplaceTempView("temp_sqlDF_SchoolInEveryYear")
sqlDF_2015_clean = spark.sql("SELECT * FROM temp_sqlDF_SchoolInEveryYear where Academic_Year=2015")

In [22]:
sqlDF_2015_clean.count()

Py4JJavaError: An error occurred while calling o70.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 64.0 failed 1 times, most recent failure: Lost task 1.0 in stage 64.0 (TID 2105) (udc-ba34-13c0 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.sql.execution.columnar.BasicColumnBuilder.initialize(ColumnBuilder.scala:66)
	at org.apache.spark.sql.execution.columnar.NativeColumnBuilder.org$apache$spark$sql$execution$columnar$NullableColumnBuilder$$super$initialize(ColumnBuilder.scala:97)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.initialize(NullableColumnBuilder.scala:51)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.initialize$(NullableColumnBuilder.scala:42)
	at org.apache.spark.sql.execution.columnar.NativeColumnBuilder.org$apache$spark$

In [ ]:
#Count each column from the merged data frame
from pyspark.sql.functions import col, when, count
df2 = sqlDF_2010_clean.select([count(when(~col(c).contains('None') & \
                            ~col(c).contains('NULL') & \
                            (col(c) != '' ) & \
                            ~col(c).isNull(), c 
                           )).alias(c)
                    for c in sqlDF_2010_clean.columns])
df2.show(2)